📩 SMS Spam Detection using Machine Learning

Spam messages continue to flood our inboxes, posing not only a nuisance but also potential security risks such as phishing attacks and scams. In this project, we explore a data-driven approach to detect and classify SMS messages as spam or ham (non-spam) using machine learning techniques.

We will be working with the SMS Spam Collection Dataset, which contains thousands of real-world text messages labeled as either spam or ham. The goal is to build a reliable classification model that can automatically identify spam messages based on the text content.

We shall start by loading the dataset and exploring it

In [ ]:
import pandas as pd

# Load the dataset and display the first 5 rows

df = pd.read_csv(r'C:\Users\user\Documents\Data-Science-Projects\SMS-Spam-Detection\data\spam.csv', encoding='latin-1')


print(df.head())


     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
